In [230]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [231]:
dataset = pd.read_csv("breast-cancer-wisconsin.data.csv")
dataset.replace('?', np.nan, inplace=True)
dataset.dropna(axis = 1,inplace=True)

In [232]:
datatype = dataset.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())
newdata = []
for data in datatype:
    newdata.append(datatype[1])
datatype = newdata
datatype

[True, True, True, True, True, True, True, True, True, True]

In [233]:
# datatype = dataset.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())
# newdata = []
# for data in datatype:
#     newdata.append(datatype[1])
# datatype = newdata
# datatype

> Training


In [234]:
# sc = StandardScaler()
# param = {'kernel': ['poly', 'linear'], 'C': [1, 10], 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'gamma': ['scale', 'auto']}
# clf = GridSearchCV(SVC(probability=True), param, cv=5, refit=True)
# 61.5%, worse then svm

In [235]:
x = dataset.iloc[:, :-1].values
y =dataset.iloc[:,dataset.shape[1]-1].values

In [236]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=0)

In [237]:
clf = SVC(kernel='poly', degree = 4, probability=True)

In [238]:
# x = dataset.iloc[:, :-1]
# y = dataset.iloc[:,dataset.shape[1]-1]
# sc = StandardScaler()
# xtrain = sc.fit_transform(xtrain)
rs = StandardScaler()
xtrain = rs.fit_transform(xtrain)

In [239]:
clf.fit(xtrain, ytrain)

SVC(degree=4, kernel='poly', probability=True)

> Testing


In [240]:
ypred = clf.predict(xtest)
print(ypred)
print(np.matrix(ytest))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[4 2 4 4 2 2 4 2 2 2 2 2 2 2 2 2 2 2 4 4 2 4 4 4 2 4 4 4 2 2 4 2 2 2 2 2
  2 4 2 2 2 2 2 4 4 4 2 4 2 4 2 2 2 4 2 2 2 2 2 4 4 4 4 4 2 4 4 2 4 4 2 2
  4 2 2 2 2 2 4 2 4 2 2 2 2 2 4 2 2 4 4 4 2 4 2 4 2 2 2 2 4 4 4 4 2 2 2 2
  2 2 2 4 2 2 4 2 4 2 2 4 2 2 4 4 4 2 2 4 2 2 4 4 2 4 2 2 2 2 4 4 4 2 4 4
  2 2 2 4 2 2 2 4 2 4 2 2 2 2 2 4 4 2 2 2 4 2 2 2 4 2 2 2 2 4 4]]


In [245]:
print(classification_report(ytest, ypred, target_names=['Benign', 'Malignant'], zero_division=1))

              precision    recall  f1-score   support

      Benign       0.62      1.00      0.76       108
   Malignant       1.00      0.00      0.00        67

    accuracy                           0.62       175
   macro avg       0.81      0.50      0.38       175
weighted avg       0.76      0.62      0.47       175



In [242]:
cm = confusion_matrix(ytest, ypred)
print(cm)

[[108   0]
 [ 67   0]]
